# Trading Strategy based on Exponential Moving Average

## Importing necessary libraries

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import numpy as np

## Getting the data and cleaning it

In [20]:
def data_download_and_clean(ticker_):
    global data, buy_and_hold_final_amount
    data = yf.download(tickers = ticker_ ,period = 'max', interval = '1d')
    buy_and_hold_final_amount = (data.iloc[-1,4] / data.iloc[1,1])*1000   # amount if we dont trade
    data.dropna(axis = 0,inplace = True)
    data.drop(['Adj Close','Volume'], axis = 1, inplace = True)
    data.reset_index(inplace = True)

## Executing the trade

In [30]:
def trade(initial_money, i, j):
    data['EMA(1)'] = data.iloc[:,4].ewm(span=i, adjust=False).mean()
    data['EMA(2)'] = data.iloc[:,4].ewm(span=j, adjust=False).mean()
    money = initial_money
    equity_amount = 0
    trade_count = 0          # number of trades done
    trading_fees = 0.001    # binance trading fees
    traded = 0               # initially no trade has taken place .. this is a binary variable
    for i in range(1,len(data)):
        # if previous day's ma(1)>ma(2) and we have not alrady bought equity then buy equity
        if((data.iloc[i-1,5] > data.iloc[i-1,6]) and traded == 0):      
            equity_amount = (money)/data.iloc[i,1]*(1-trading_fees)     
            money = 0                                                  # set current amount to 0
            trade_count = trade_count+1                                # increase trade count
            traded = 1                                                 # change traded flag to 1
        # if previous day's ma(2) < ma(5) and we have bought the equity then sell equity
        elif((data.iloc[i-1,5] < data.iloc[i-1,6]) and traded == 1):       
            money = (equity_amount)*data.iloc[i,1]*(1-trading_fees)    
            equity_amount = 0                                          # set equity amount to 0
            trade_count = trade_count+1                                # increase trade count
            traded = 0                                                 # change traded flag to 0
        else:
            pass                                                       # if neither conditions happen then we don't do anything
        
    if(money == 0):
        final_money = equity_amount*data.iloc[i,4]  # checking our final amount
    else:
        final_money = money
    print('Profit with trading(percentage) =', 100*(final_money / initial_money - 1)) 
    print('Profit with holding(percentage) =', 100*(buy_and_hold_final_amount / initial_money - 1))
    print('Trading Strategy vs Holding profit difference = $',final_money - buy_and_hold_final_amount)
    print('Number of trades executed =', trade_count)
    #return final_money/initial_money

## Calculating the profit

In [4]:
from IPython.display import clear_output

In [15]:
max_0 = 0
max_i = 0
max_j = 0
data_download_and_clean('BTC-USD')
for i in range(1,100):
    for j in range(i+1,100):
        amount = trade(1000, i, j)
        if(amount>max_0):
            max_0 = amount
            max_i = i
            max_j = j
        clear_output(wait = True)
        print('currently running: ' , i , j)

currently running:  98 99


In [16]:
print(max_0, max_i, max_j)

343.1890875030044 36 38


Removing the commented code in trade and running it once again

In [22]:
data_download_and_clean('BTC-USD')
trade(1000, max_i, max_j)

[*********************100%***********************]  1 of 1 completed
Profit with trading(percentage) = 34467.97143068614
Profit with holding(percentage) = 12356.349855461327
Trading Strategy vs Holding profit difference = $ 221116.21575224813
Number of trades executed = 25


The above profit is for the time period 'max', i.e. starting from 17 Sept 2014 to 3 Dec 2021. with 1d timeframe

In [23]:
data

,Date,Open,High,Low,Close,EMA(1),EMA(2)
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,457.334015
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,455.555960,455.647142
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,452.271637,452.526571
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,449.927440,450.289515
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,447.164931,447.650105
...,...,...,...,...,...,...,...
2625,2021-11-28,54813.023438,57393.843750,53576.734375,57248.457031,59023.722923,58959.104369
2626,2021-11-29,57291.906250,58872.878906,56792.527344,57806.566406,58957.930679,58899.999858
2627,2021-11-30,57830.113281,59113.402344,56057.281250,57005.425781,58852.389874,58802.842213
2628,2021-12-01,56907.964844,59041.683594,56553.082031,57229.828125,58764.683833,58722.174824


___

Now, I will compute the profit for 1 yr on 1hr timeframe

In [25]:
def data_download_and_clean2(ticker_):
    global data, buy_and_hold_final_amount
    data = yf.download(tickers = ticker_ , period = '1y' , interval = '1h')
    buy_and_hold_final_amount = (data.iloc[-1,4] / data.iloc[1,1])*1000   # amount if we dont trade
    data.dropna(axis = 0,inplace = True)
    data.drop(['Adj Close','Volume'], axis = 1, inplace = True)
    data.reset_index(inplace = True)

In [28]:
max_0 = 0
max_i = 0
max_j = 0
data_download_and_clean2('BTC-USD')
for i in range(1,100):
    for j in range(i+1,100):
        amount = trade(1000, i, j)
        if(amount>max_0):
            max_0 = amount
            max_i = i
            max_j = j
        clear_output(wait = True)
        print('currently running: ' , i , j)
clear_output(wait = True)
print('Training Complete')

Training Complete


In [29]:
print(max_0, max_i, max_j)

4.0419936217796515 3 79


In [31]:
data_download_and_clean('BTC-USD')
trade(1000, max_i, max_j)

[*********************100%***********************]  1 of 1 completed
Profit with trading(percentage) = 15299.463716106138
Profit with holding(percentage) = 12360.64804733184
Trading Strategy vs Holding profit difference = $ 29388.156687742972
Number of trades executed = 61
